Please save your files in a directory called *Assignments*

Name your files: *A1P1.ipynb* and *A1P2.ipynb* for problems 1 and 2 of Assignment 1. 

# Problem 1

(1) Consider the cubic polynomial $z^3-1$.  The roots are $1, e^{2 \pi i/3}$ and $e^{-2 \pi i /3}$. Write a python function that takes as input four floting point numbers $x_0, x_1, y_0, y_1$ and an integer $n$, and plots (on the screen) which root Newton's method converges to, when one uses an initial guess in a grid of point in the rectangle $[x_0, x_1] \times [y_0, y_1]$, using resolution $n$ for the grid.  Use colours to indicate which root Newton's method finds.  Include a plot for the rectangle $[-2,2] \times [-2,2]$ with $n=400$.  I have included a symbolic version of Newton's method that allows you to use complex numbers as input, it is a slight modification of the algorithm from the second week.

In [3]:
## Newton's method that allows for complex number input.
from sympy import *

sym_x=Symbol("x")

def OI_Newton( f ):
    g = sym_x - f/diff(f,sym_x)
    return g

Sym_func = sym_x**3 - 1

Newt_Iterator = OI_Newton(Sym_func)

from mpmath import *

mp.dps = 20
mp.prec = mp.dps * 3.34
mp.pretty = True

NIt = lambdify(sym_x, Newt_Iterator, "mpmath")

## Here is an example of how Newton's method for z^3 - 1 converges to two different
## roots, depending on initial condition.

z = mpc(1.0, 0.5) ## mpc is the mpmath arbitrary-precision complex number floating-point type.
print "Initial condition: ", z

for i in range(0,5):
    Z = z
    z = NIt(z)
    print z

print("Evaluating ", Sym_func, " at ", z, " gives: ", Sym_func.evalf(subs = {sym_x : z}))
    
z = mpc(-1,0.5)
print("Initial condition: ", z)

for i in range(0,5):
    Z = z
    z = NIt(z)
    print z

print("Evaluating ", Sym_func, " at ", z, " gives: ", Sym_func.evalf(subs = {sym_x : z}))

print("Both are very close to zero.")

Initial condition:  (1.0 + 0.5j)
(0.8266666666666666667 + 0.12j)
(1.009101293268602287 - 0.05582701123777840886j)
(0.9970913786920296195 - 0.001218678160774645252j)
(1.000006990428805843 + 7.128339109188268049e-6j)
(0.9999999999980538418 + 9.96593837081526383e-11j)
('Evaluating ', x**3 - 1, ' at ', (0.9999999999980538418 + 9.96593837081526383e-11j), ' gives: ', -5.8383298493809e-12 + 2.98978151123294e-10*I)
('Initial condition: ', (-1.0 + 0.5j))
(-0.5066666666666666667 + 0.5466666666666666667j)
(-0.3832817777777777778 + 0.9627164444444444444j)
(-0.4810173976123450126 + 0.8551826783058666047j)
(-0.5004900329923770473 + 0.8660135028433907189j)
(-0.50000010998239533 + 0.8660251902444721468j)
('Evaluating ', x**3 - 1, ' at ', (-0.50000010998239533 + 0.8660251902444721468j), ' gives: ', -3.89819586060623e-7 + 6.06052437207287e-7*I)
Both are very close to zero.


# Problem 2

(2) In class we have seen implementations of the mid-point rule and Simpson's 3/8-method to approximate integrals. One of the key features of these methods (especially useful when your function comes from measured data) is that the technique uses as its input **only** the values of $f(x)$ at a finite number of points.  Sometimes one has access to **more** information about $f$ than simply its values.  In this problem, we ask you to write a script to approximate integrals numerically, but where one has a **symbolic** expression for $f$, and therefore using sympy, one also has access to $f'$. 

**Fact** Given an interval $[a,b]$ and numbers $f_a, f'_a, f_b, f'_b$ there exists one (and only one) cubic polynomial $p(x) = c_0 + c_1 x + c_2 x^2 + c_3 x^3$ such that $$p(a) = f_a, \ \ p'(a) = f'_a, \ \ p(b) = f_b, \ \ p'(b) = f'_b$$

Moreover, one can solve for $c_0,c_1,c_2,c_3$ fairly easily.  The above equations are:

$$c_0 + c_1 a + c_2 a^2 + c_3 a^3 = f_0 \ [eq.1]\ \ c_0 + c_1 b + c_2 b^2 + c_3 b^3 = f_1 \ [eq.2] $$
$$c_1 + 2c_2a + 3c_3a^2 = f'_0 \ [eq.3] \ \ c_1 + 2c_2b+3c_3b^2 = f'_1 \ [eq.4] $$

a little manipulation and one gets to the expression:

$$\pmatrix{ c_2 \\ c_3} = \frac{1}{(a-b)^3} \cdot \pmatrix{ 3(b+a) & -2(b^2+ba+a^2) \\ -2 & b+a} \cdot \pmatrix{ f_0 - f_1 + bf_1' - af_0' \\ f_1' -f_0'}$$

the middle expression is a $2 \times 2$ matrix. From the above one can solve for $c_0$ and $c_1$ by back-substituting into [eq.1] and [eq.3] (or [eq.2] and [eq.4], your choice). 

Compare the accuracy of this method to the mid-point and Simpson 3/8-method, for the computation of the integral $\int_0^\pi \sin(x) dx = 2$. Try the number of intervals $n=10, 100, 1000$ for all three methods. 

Aside from accuracy, what other considerations would cause you to prefer one method over the other two?

In [4]:
def midpt_integral(f, I, k): 
    sum = 0
    deltax = (I[1]-I[0])/float(k)
    for i in range (0, k):
        x = ((i/float(k))*I[1]) + ((1.0-(i/float(k)))*I[0]) + deltax/2
        sum = sum + f(x)*deltax
    return sum


def threeeight_integral(f, I, k): 
    sum = 0
    deltax = (I[1]-I[0])/k
    for i in range (0, k):
        xa = ((i/float(k))*I[1]) + ((1-i/float(k))*I[0]) 
        xb = ((i/float(k))*I[1]) + ((1-i/float(k))*I[0]) + deltax/3 
        xc = ((i/float(k))*I[1]) + ((1-i/float(k))*I[0]) + (2*deltax)/3
        xd = ((i/float(k))*I[1]) + ((1-i/float(k))*I[0]) + deltax
        sum = sum + ( f(xa) + 3*f(xb) + 3*f(xc) + f(xd) )*deltax/8
    return sum



In [5]:
subdivisions = 20
## Ensure your cubic_spline_int function is callable as below. 
#print "Cubic spline with 10 subdivisions: ", cubic_spline_int(sin(sym_x), [mpf('0'), pi], subdivisions), "Error: ", 2.0-cubic_spline_int(sin(sym_x), [mpf('0'), pi], subdivisions)
print("Midpoint with 10 subdivisions: ", midpt_integral(sin, [0.0, pi], subdivisions), "Error ", 2.0 - midpt_integral(sin, [0.0, pi], subdivisions))
print("Three eights integral: ", threeeight_integral(sin, [0.0, pi], subdivisions/3), "Error ", 2.0 - threeeight_integral(sin, [0.0, pi], subdivisions/3))


('Midpoint with 10 subdivisions: ', 2.002057648285417056, 'Error ', -0.002057648285417055633)
('Three eights integral: ', 2.000023367367170042, 'Error ', -0.00002336736717004186991)
